In [1]:
%autosave 300
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [5]:
import os
import json

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/Abi_GenAI_Sessions"
)
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/copilot-model-run/code/Users/Soutrik.Chowdhury/Abi_GenAI_Sessions'

In [6]:
# read a json file from .kaggle and set the environment variables
# os.environ['KAGGLE_USERNAME'] = 'your_kaggle_username'
# os.environ['KAGGLE_KEY'] = 'your_kaggle_api_key'

with open(".config/kaggle.json") as f:
    kaggle_json = json.load(f)

os.environ["KAGGLE_USERNAME"] = kaggle_json["username"]
os.environ["KAGGLE_KEY"] = kaggle_json["key"]

In [7]:

import kaggle

In [8]:
if not os.path.exists("./data/taylor-swift-all-lyrics-30-albums.zip"):
    kaggle.api.dataset_download_files(
        "ishikajohari/taylor-swift-all-lyrics-30-albums",
        path="./data",
        unzip=True,
    )

In [9]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import (
    get_peft_model,
    PeftConfig,
    PeftModel,
    LoraConfig,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    GenerationConfig,
)
from trl import SFTTrainer
from datasets import Dataset

2024-09-25 10:32:48.900841: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-25 10:32:48.900889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-25 10:32:48.901780: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-25 10:32:48.908044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-25 10:32:49.623287: W tensorflow/compiler/tf2

Initial data processing

In [10]:
data_path = "data/data/Albums"
lyrics_files = glob.glob(f"{data_path}/**/*.txt", recursive=True)
print(len(lyrics_files))

879


In [11]:
def read_and_join_txt_files(file_paths):
    contents = []
    for path in file_paths:
        with open(path, "r", encoding="utf-8") as file:
            contents.append(file.read())
    return "\n\n".join(contents)

In [12]:
joined_content = read_and_join_txt_files(lyrics_files)

In [13]:
print(joined_content[:1000])

104 ContributorsTranslationsEspañolPortuguêsFrançaisClean Lyrics[Verse 1]
The drought was the very worst (Oh-oh, oh-oh)
When the flowers that we'd grown together died of thirst
It was months and months of back and forth (Oh-oh, oh-oh)
You're still all over me
Like a wine-stained dress I can't wear anymore

[Pre-Chorus]
Hung my head as I lost the war
And the sky turned black like a perfect storm
[Chorus]
Rain came pouring down
When I was drowning, that's when I could finally breathe
And by morning
Gone was any trace of you, I think I am finally clean
(Oh, oh, oh, oh)

[Verse 2]
There was nothing left to do (Oh-oh, oh-oh)
When the butterflies turned to
Dust that covered my whole room
So I punched a hole in the roof (Oh-oh, oh-oh)
Let the flood carry away all my pictures of you

[Pre-Chorus]
The water filled my lungs, I screamed so loud
But no one heard a thing

[Chorus]
Rain came pouring down
When I was drowning, that's when I could finally breathe
And by morning
Gone was any trace of yo

In [14]:
# Cleaning the file by removing/replacing unnecessary characters and removing sections
# that are not lyrics
replace_with_space = ["\u2005", "\u200b", "\u205f", "\xa0", "-"]
replace_letters = {
    "í": "i",
    "é": "e",
    "ï": "i",
    "ó": "o",
    ";": ",",
    "‘": "'",
    "’": "'",
    ":": ",",
    "е": "e",
}
remove_list = ["\)", "\(", "–", '"', "”", '"', "\[.*\]", ".*\|.*", "—"]

cleaned_lyrics = joined_content

for old, new in replace_letters.items():
    cleaned_lyrics = cleaned_lyrics.replace(old, new)
for string in remove_list:
    cleaned_lyrics = re.sub(string, "", cleaned_lyrics)
for string in replace_with_space:
    cleaned_lyrics = re.sub(string, " ", cleaned_lyrics)
print("".join(sorted(set(cleaned_lyrics))))

	
 !#$&'*,./0123456789?ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz¦ÃÅÇÜáâãåçêñöüăčšΕάήαεηικλμνρστφМРСУагдийклнопрсуьїابةتجرسعفلمهيیदनहिी्ทปภยลษาแไᜄᜆᜎᜓ᜔ếệ‌‎“†…中文日本翻語译국어한


In [15]:
# Setting aside a portion for training the model and a portion for testing the data to prevent
# the model from overfitting to the data it is tested on
split_point = int(len(cleaned_lyrics) * 0.95)
train_data = cleaned_lyrics[:split_point]
test_data = cleaned_lyrics[split_point:]
train_data_seg = []
for i in range(0, len(train_data), 500):
    text = train_data[i : min(i + 500, len(train_data))]
    train_data_seg.append(text)
train_data_seg = Dataset.from_dict({"text": train_data_seg})
print(len(train_data_seg))

3730


In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

In [18]:
# Loading the model with double quantization and 4-bit quantization for the model weights and activations
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [19]:
# Loading the model with the quantization configuration and setting the device map to auto
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

In [20]:
#  Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [21]:
# Generating lyrics with the base model. The repetition penalty in the generation config prevents the model from continually repeating the same string.
device = "cuda" if torch.cuda.is_available() else "cpu"


def generate_lyrics(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(
        max_new_tokens=250,
        pad_token_id=tokenizer.eos_token_id,
        repetition_penalty=1.3,
        eos_token_id=tokenizer.eos_token_id,
    )
    outputs = model.generate(
        input_ids=encoding.input_ids, generation_config=generation_config
    )
    text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("INPUT\n", query, "\n\nOUTPUT\n", text_output[len(query) :])

In [22]:
generate_lyrics(test_data[200:700], model)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


INPUT
 , so take me to Florida
I've got some regrets, I'll bury them in Florida
Tell me I'm despicable, say it's unforgivable
What a crash, what a rush, fuck me up, Florida
It's one hell of a drug
It's one hell of a drug
Love left me like this and I don't want to exist
So take me to Florida

Little did you know your home's really only
A town you're just a guest in Florida
So you work your life away just to pay
For a time share down in Destin Take me to Florida
Little did you know your home's really onl 

OUTPUT
 
You can get the best of everything
But if you do not have enough money for rent
Then you are going to be living out there
And that is where we all live now
We all live here together

Now let us go back to our first song
The second song was called "Florida"
This is my favorite song from the album
My favorite song from the album

Oh yeah, oh yeah, oh yeah
Well, well, well, well
Well, well, well, well
Well, well, well, well
Well, well, well, well
Well, well, well, well
Well, well

In [23]:
# Setting arguments for low-rank adaptation 

model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05 
lora_rank = 32 

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

In [24]:
# Setting training arguments

output_dir = "tommyadams/tinyllama"  # Model repo on your hugging face account where you want to save your model
per_device_train_batch_size = 3
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_strategy = "steps"
save_steps = 10
logging_steps = 10
learning_rate = 2e-3
max_grad_norm = 0.3  # Sets limit for gradient clipping
max_steps = 50  # Number of training steps
warmup_ratio = 0.03  # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine"  # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to="none",
)

In [25]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_data_seg,
    peft_config=peft_config,
    max_seq_length=500,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

/anaconda/envs/finetune_env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/anaconda/envs/finetune_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/anaconda/envs/finetune_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3730 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [26]:
trainer.train()

/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.972800
20,2.849100
30,2.752500
40,2.616600
50,2.703500


/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/anaconda/envs/finetune_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/anaconda/envs/finetune_env/lib/python3.10/site-packages

TrainOutput(global_step=50, training_loss=2.778901176452637, metrics={'train_runtime': 160.7288, 'train_samples_per_second': 1.866, 'train_steps_per_second': 0.311, 'total_flos': 320408209833984.0, 'train_loss': 2.778901176452637, 'epoch': 0.08038585209003216})

In [3]:
generate_lyrics(test_data[200:300], peft_model)

NameError: name 'generate_lyrics' is not defined

###################################################################### END ######################################################################